In [53]:
import pandas as pd
import numpy as np

In [54]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
df=pd.read_csv('test.csv')

In [56]:
df

,text,label
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,0
3,i left with my bouquet of red and yellow tulip...,1
4,i was feeling a little vain when i did this one,0
...,...,...
1995,i just keep feeling like someone is being unki...,3
1996,im feeling a little cranky negative after this...,3
1997,i feel that i am useful to my people and that ...,1
1998,im feeling more comfortable with derby i feel ...,1


In [57]:
df.shape

(2000, 2)

In [58]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [59]:
df.duplicated().sum()

0

In [60]:
df.drop_duplicates(inplace=True)

In [61]:
df.duplicated().sum()

0

In [62]:
# Basic Preprocessing
# Lowercase
# Remove Stopwords
# Stemming

In [63]:
import re

In [64]:
def remove_tags(raw_text):
    cleaned_text= re.sub(re.compile('<.*?>'),"",raw_text)
    return cleaned_text

In [65]:
df['text']= df['text'].apply(remove_tags)

In [66]:
df['text']

0       im feeling rather rotten so im not very ambiti...
1               im updating my blog because i feel shitty
2       i never make her separate from me because i do...
3       i left with my bouquet of red and yellow tulip...
4         i was feeling a little vain when i did this one
                              ...                        
1995    i just keep feeling like someone is being unki...
1996    im feeling a little cranky negative after this...
1997    i feel that i am useful to my people and that ...
1998    im feeling more comfortable with derby i feel ...
1999    i feel all weird when i have to meet w people ...
Name: text, Length: 2000, dtype: object

In [67]:
import nltk

In [68]:
from nltk.corpus import stopwords
sw_list=stopwords.words('english')
df["text"]= df["text"].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [69]:
df

,text,label
0,im feeling rather rotten im ambitious right,0
1,im updating blog feel shitty,0
2,never make separate ever want feel like ashamed,0
3,left bouquet red yellow tulips arm feeling sli...,1
4,feeling little vain one,0
...,...,...
1995,keep feeling like someone unkind wrong think g...,3
1996,im feeling little cranky negative doctors appo...,3
1997,feel useful people gives great feeling achieve...,1
1998,im feeling comfortable derby feel though start...,1


In [70]:
from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split() ])

In [71]:
df["text"]= df["text"].apply(stem_words)

In [72]:
df

,text,label
0,im feel rather rotten im ambiti right,0
1,im updat blog feel shitti,0
2,never make separ ever want feel like asham,0
3,left bouquet red yellow tulip arm feel slightl...,1
4,feel littl vain one,0
...,...,...
1995,keep feel like someon unkind wrong think get b...,3
1996,im feel littl cranki neg doctor appoint,3
1997,feel use peopl give great feel achiev,1
1998,im feel comfort derbi feel though start step s...,1


In [73]:
X= df.iloc[:,0:1]
y=df['label']

In [74]:
X

,text
0,im feel rather rotten im ambiti right
1,im updat blog feel shitti
2,never make separ ever want feel like asham
3,left bouquet red yellow tulip arm feel slightl...
4,feel littl vain one
...,...
1995,keep feel like someon unkind wrong think get b...
1996,im feel littl cranki neg doctor appoint
1997,feel use peopl give great feel achiev
1998,im feel comfort derbi feel though start step s...


In [75]:
y

0       0
1       0
2       0
3       1
4       0
       ..
1995    3
1996    3
1997    1
1998    1
1999    4
Name: label, Length: 2000, dtype: int64

In [76]:
import sklearn

In [77]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y= encoder.fit_transform(y)

In [78]:
y

array([0, 0, 0, ..., 1, 1, 4], dtype=int64)

In [79]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.1,random_state=1)

In [80]:
X_train.shape

(1800, 1)

In [81]:
# Applying BoW
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [82]:
X_train_bow= cv.fit_transform(X_train["text"]).toarray()
X_test_bow= cv.transform(X_test["text"]).toarray()

In [83]:
X_train_bow.shape

(1800, 3380)

In [84]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [85]:
# Algorithm 2 (Random Forest)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred= rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.73

In [97]:
def predict_sentiment(text):
    text = remove_tags(text)
    text2 = text.lower()
    stemmed_sent= stem_words(text2) 
    text_bow = cv.transform([stemmed_sent]).toarray()
    prediction = rf.predict(text_bow)
    sentiment = encoder.inverse_transform(prediction)[0]
    if sentiment == 0:
        return 'Sadness'    
    elif sentiment == 1:
        return 'Joy'
    elif sentiment == 2:
        return 'Love'
    elif sentiment == 3:
        return 'Anger'
    else: 
        return 'Fear'

# Example usage
predict_sentiment("God give us strengths")

'Anger'

In [87]:
confusion_matrix(y_test,y_pred)

array([[51, 13,  0,  1,  1,  0],
       [ 2, 63,  4,  1,  2,  3],
       [ 0,  7,  1,  0,  1,  0],
       [ 5,  4,  1, 11,  1,  0],
       [ 1,  1,  0,  2, 17,  1],
       [ 0,  1,  0,  0,  2,  3]], dtype=int64)

In [88]:
# # Algorithm 1 (Gaussian Naive Bayes)
# from sklearn.naive_bayes import GaussianNB
# gnb= GaussianNB()

# gnb.fit(X_train_bow,y_train)

# y_pred= gnb.predict(X_test_bow)
# from sklearn.metrics import accuracy_score,confusion_matrix
# accuracy_score(y_test,y_pred)

In [89]:
import pickle
pickle.dump(rf,open('model.pkl','wb'))
pickle.dump(cv,open('cv.pkl','wb'))